In [54]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC
import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

In [4]:
dataDict = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
print (len(dataDict))

7681


In [5]:
keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_Naver.pickled','rb'))

In [6]:
for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

In [7]:
df = pd.DataFrame.from_dict(dataDict, orient='index')
df['date'] = pd.to_datetime(df['date']).dt.date
df.reset_index(inplace = True)
df.rename(columns={'index':'id'}, inplace=True)
print (df.shape)
df.head()

(7681, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a29c445588c132954d1973a,정치,2017-12-07,연합뉴스,1713,1474,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","[외무성, 핵전쟁, 대변인]","{대변인, 조선반도, 핵전쟁, 중앙, 고위, 북한, 미국, 도화선}"
1,5a29c445588c132954d1973b,정치,2017-12-07,한국일보,2551,2121,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"[예산, 예결위, soc]","{정부안, 증액, 호남, 의원, 지역구, 국민의당, 예산안}"
2,5a29c445588c132954d1973c,정치,2017-12-07,뉴시스,610,540,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"[최경환, 구속영장, 국가정보원]","{구속영장 청구, 검찰, 정기국회, 혐의, 의원, 조사, 국정원장}"
3,5a29c445588c132954d1973d,정치,2017-12-07,연합뉴스,145,134,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"[이슈 · 최재형 감사원장 내정, 감사원장, 최재형, 감사원]","{법관, 생활, 지명, 감사원장, 후보자, 공직 사회}"
4,5a29c445588c132954d1973e,정치,2017-12-07,동아일보,1074,937,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"[김정은, b-1b, 한반도]","{김정은, 펠트먼, 접경, 양강도, 삼지연, 공장, 훈련, 사무차장, 시찰}"


In [8]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [9]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    res = [x for x in words if len(x) != 1]
    return res

In [10]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [11]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [12]:
w2v_docs = list()
for idx in tqdm(df.index):
    text = df.loc[idx,'title']+'.\n'+df.loc[idx,'mainText']
    pos = nav_tokenizer(text)
    pos = remove_stopwords(pos, stopwords)
    label = df.loc[idx, 'category']
    w2v_docs.append(TaggedDocument(pos, label))

100%|██████████| 7681/7681 [06:50<00:00, 18.71it/s]


In [59]:
train, test = train_test_split(w2v_docs, test_size = 0.15)

In [60]:
x_train = [ x.words for x in train] 
y_train = [ x.tags for x in train] 
x_test = [ x.words for x in test] 
y_test = [ x.tags for x in test] 

## word2vec

In [17]:
w2v_model = Word2Vec(sg=1, size=3000,window=8, workers = 6, iter = 20)
w2v_model.build_vocab(tqdm(x_train))
w2v_model.train(tqdm(x_train), total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)

100%|██████████| 6528/6528 [02:20<00:00, 46.61it/s] 


30418110

In [79]:
w2v_model.save('./model/keywords_classification_naver_by_word2vec_size3000_window8_iter20_by_mecab.model')

In [15]:
w2v_model = Word2Vec.load('./model/keywords_classification_naver_by_word2vec_size3000_window8_iter20_by_mecab.model')

In [16]:
print(w2v_model.most_similar('신혜선'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('박시후', 0.7428619265556335), ('서은수', 0.7016558647155762), ('서지안', 0.6734795570373535), ('사직서', 0.6398009657859802), ('이태환', 0.6336354613304138), ('선우혁', 0.6287235021591187), ('해성그룹', 0.622657060623169), ('유인영', 0.5950829982757568), ('천호진', 0.5877219438552856), ('파혼', 0.5856837034225464)]


In [17]:
print(w2v_model.most_similar('손흥민'))

[('토트넘', 0.6116730570793152), ('요렌테', 0.5742184519767761), ('스토크시티', 0.5558681488037109), ('라멜라', 0.5413873791694641), ('아포엘', 0.5385053157806396), ('에릭센', 0.5334597826004028), ('델레', 0.5244540572166443), ('지전', 0.523140549659729), ('군면제', 0.5226224660873413), ('사우스햄튼', 0.515069842338562)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [18]:
print(w2v_model.most_similar('안철수'))

[('국민의당', 0.596470296382904), ('재신임', 0.5626682639122009), ('박지원', 0.5384296774864197), ('좌고우면', 0.5348546504974365), ('반안', 0.5297796726226807), ('용팔이', 0.5293495655059814), ('대표직', 0.527755618095398), ('선관위원장', 0.5194007158279419), ('가처분신청', 0.5146774053573608), ('친안', 0.5094169974327087)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [19]:
print(w2v_model.most_similar('이명박'))

[('박근혜정부', 0.4172804057598114), ('박연차', 0.41612884402275085), ('실소유', 0.41514158248901367), ('국교단절', 0.4144594371318817), ('원전수주', 0.41435617208480835), ('박근혜', 0.40976226329803467), ('조갑제', 0.402931809425354), ('동영', 0.4013504981994629), ('실소유주', 0.39642438292503357), ('성명불상', 0.39121320843696594)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [20]:
print(w2v_model.most_similar('문재인'))

[('서미선', 0.422291100025177), ('배재만', 0.402286171913147), ('케어', 0.4005219340324402), ('수석보좌관', 0.4003939926624298), ('문재', 0.3971909284591675), ('영빈관', 0.3833824396133423), ('의료수가', 0.3832871913909912), ('대통령', 0.38195836544036865), ('박경준', 0.37973758578300476), ('김태규', 0.3731063902378082)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [21]:
print(w2v_model.most_similar('박근혜'))

[('홍용표', 0.42260444164276123), ('남재준', 0.4213549494743347), ('이보배', 0.4190545678138733), ('친박', 0.410078763961792), ('이명박', 0.40976226329803467), ('이병호', 0.40735989809036255), ('최순실', 0.4056743383407593), ('현기환', 0.4051252007484436), ('국정농단', 0.40346211194992065), ('이원종', 0.4028346538543701)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform(x_train)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 10893


In [23]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [24]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

0it [00:00, ?it/s]/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
6528it [02:06, 51.64it/s]
1153it [00:30, 38.33it/s]


In [28]:
w2v_model.save('./model/word2vec_category_size3000_window8_iter20_by_mecab.w2v')

In [ ]:
w2v_model = Word2Vec.load('./model/word2vec_category_size3000_window8_iter20_by_mecab.w2v')

In [25]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [26]:
le.fit(y_train)
le.classes_

array(['IT/과학', '경제', '사회', '생활/문화', '세계', '스포츠', '연예', '정치'], dtype='<U5')

In [27]:
from keras.utils import np_utils
y_test2 = np_utils.to_categorical(le.transform(y_test),8)
y_train2 = np_utils.to_categorical(le.transform(y_train),8)

In [28]:
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.layers import Input, Dense, Embedding, embeddings, merge, Dropout
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=3000))
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(8, activation='sigmoid'))
model.compile(#optimizer='rmsprop',
    optimizer='adadelta',
              #loss='binary_crossentropy',
    #optimizer=SGD(lr=0.2), 
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train2, epochs=100, verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
score = model.evaluate(test_vecs_w2v, y_test2, verbose=2)
print (score[1])

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=6528), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 96', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 97', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 98', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 99', max=6528), HTML(value='')))


0.9150043365134432


In [29]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
model2 = Sequential()
model2.add(Dense(64, activation='relu', input_dim=3000))
model2.add(Dropout(0.25))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.125))
model2.add(Dense(8, activation='softmax'))
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
model2.fit(train_vecs_w2v, y_train2,
          epochs=100, verbose=0,
          callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
score = model2.evaluate(test_vecs_w2v, y_test2)
print (score[1])

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=6528), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 96', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 97', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 98', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 99', max=6528), HTML(value='')))


1153/1153 [==============================] - 0s 221us/step
0.9054640069384216
